In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm

%load_ext watermark
%load_ext lab_black

# Dugongs
This example is the first example of dealing with missing data.

It is adapted from [unit 6: dugongsmissing.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/dugongsmissing.odc).

## Associated lecture video: Unit 6 Lesson 6

%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=57" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement
Carlin and Gelfand (1991) investigated the age (x) and length (y) of 27 captured dugongs (sea cows). Estimate parameters in a nonlinear growth model.

### References

Data provided by Ratkowsky (1983).

Carlin, B. and Gelfand, B. (1991). An Iterative Monte Carlo Method for 
Nonconjugate Bayesian Analysis, Statistics and Computing, 1, (2), 119†128.

Ratkowsky, D. (1983). Nonlinear regression modeling: A unified practical approach. M. Dekker, NY, viii, 276 p.

In [2]:
# fmt: off
X = [1.0, 1.5, 1.5, 1.5, 2.5, 4.0, 5.0, 5.0, 7.0, 8.0, 8.5, 9.0, 9.5, 
     9.5, 10.0, 12.0, 12.0, 13.0, 13.0, 14.5, 15.5, 15.5, 16.5, 17.0,
     22.5, 29.0, 31.5]
y = [1.80, 1.85, 1.87, -1, 2.02, 2.27, 2.15, 2.26, 2.47, 2.19, 2.26,
     2.40, 2.39, 2.41, 2.50, 2.32, 2.32, 2.43, 2.47, 2.56, 2.65, 2.47,
     2.64, 2.56, 2.70, 2.72, -1]
# fmt: on

PyMC imputes missing data automatically, similar to BUGS, but it requires the missing data be input as a NumPy masked array. See the NumPy docs for [np.ma.masked_values()](https://numpy.org/doc/stable/reference/generated/numpy.ma.masked_values.html). For ```y```, above, you can enter whatever number at the missing data positions, then plug it into the ```value``` parameter below. Just make sure the number you're using isn't actually in the data. I chose -1 above.

Note that this method only works for missing *observed* data. We'll learn about missing data in X later.

In [4]:
y = np.ma.masked_values(y, value=-1)
y

masked_array(data=[1.8, 1.85, 1.87, --, 2.02, 2.27, 2.15, 2.26, 2.47,
                   2.19, 2.26, 2.4, 2.39, 2.41, 2.5, 2.32, 2.32, 2.43,
                   2.47, 2.56, 2.65, 2.47, 2.64, 2.56, 2.7, 2.72, --],
             mask=[False, False, False,  True, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False,  True],
       fill_value=-1.0)

Check it out! The array now has a mask attribute.

In [6]:
with pm.Model() as m:
    # priors
    alpha = pm.Uniform("alpha", 0, 100)
    beta = pm.Uniform("beta", 0, 100)
    gamma = pm.Uniform("gamma", 0, 1)
    sigma = pm.math.exp(pm.Uniform("sigma", -10, 10))

    mu = alpha - beta * gamma**X

    likelihood = pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y)

    # start sampling
    trace = pm.sample(
        10000,
        chains=4,
        tune=500,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1325: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/aesaraf.py:996: UserWarning: The parameter 'updates' of aesara.function() expects an OrderedDict, got <class 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (collections.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
  aesara_function = aesara.function(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, gamma, sigma, likelihood_missing]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 500 tune and 10_000 draw iterations (2_000 + 40_000 draws total) took 44 seconds.
There were 24 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `t

In [7]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
likelihood_missing[0],1.904,0.110,1.684,2.116,0.001,0.001,16516.0,21145.0,1.0
likelihood_missing[1],2.692,0.127,2.435,2.932,0.002,0.001,7274.0,6980.0,1.0
alpha,2.738,0.139,2.517,3.000,0.003,0.002,4447.0,2155.0,1.0
beta,0.990,0.116,0.785,1.204,0.002,0.001,5218.0,2215.0,1.0
gamma,0.888,0.035,0.818,0.952,0.000,0.000,4975.0,2796.0,1.0
sigma,-2.341,0.154,-2.638,-2.039,0.002,0.001,5704.0,3755.0,1.0


This output is very close to the BUGS results if you use the inits labeled Alternative (full) inits (initializing the missing values at 1):

|           | mean    | sd      | MC_error | val2.5pc | median  | val97.5pc | start | sample |
|-----------|---------|---------|----------|----------|---------|-----------|-------|--------|
| alpha     | 2.731   | 0.1206  | 0.003167 | 2.551    | 2.711   | 3.025     | 1001  | 100000 |
| beta      | 0.9846  | 0.1021  | 0.002008 | 0.8053   | 0.9773  | 1.212     | 1001  | 100000 |
| gamma     | 0.8874  | 0.03381 | 7.64E-04 | 0.8124   | 0.8908  | 0.9427    | 1001  | 100000 |
| log.sigma | -2.342  | 0.1543  | 7.83E-04 | -2.622   | -2.349  | -2.018    | 1001  | 100000 |
| sigma     | 0.09733 | 0.0155  | 7.96E-05 | 0.07267  | 0.09547 | 0.1329    | 1001  | 100000 |
| y[4]      | 1.906   | 0.1098  | 6.23E-04 | 1.689    | 1.906   | 2.123     | 1001  | 100000 |
| y[27]     | 2.69    | 0.1255  | 0.001916 | 2.446    | 2.689   | 2.941     | 1001  | 100000 |


If you use the first set of inits, as Professor Vidakovic did in the video, BUGS gives the y[4] mean as 2.047 and y[27] as 3.04. All the other variables are very different here, too. This is quite a mystery - as far as I can tell, the only difference is that the missing values use BUGS-generated inits for the following results. If someone figures out what's going on here, let me know!

|       | mean   | sd       | MC_error | val2.5pc | median | val97.5pc | start | sample |
|-------|--------|----------|----------|----------|--------|-----------|-------|--------|
| alpha | 32.54  | 3.698    | 0.2076   | 23.51    | 32.69  | 40.31     | 1001  | 100000 |
| beta  | 30.55  | 3.697    | 0.2076   | 21.52    | 30.69  | 38.31     | 1001  | 100000 |
| gamma | 0.9989 | 2.08E-04 | 8.70E-06 | 0.9984   | 0.9989 | 0.9992    | 1001  | 100000 |
| sigma | 0.1328 | 0.02071  | 8.17E-05 | 0.09964  | 0.1303 | 0.1805    | 1001  | 100000 |
| y[4]  | 2.047  | 0.1424   | 5.18E-04 | 1.766    | 2.046  | 2.329     | 1001  | 100000 |
| y[27] | 3.04   | 0.1606   | 7.42E-04 | 2.722    | 3.04   | 3.358     | 1001  | 100000 |

In [8]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.3.0

numpy     : 1.22.3
pymc      : 4.0.0b5
matplotlib: 3.5.2
arviz     : 0.12.1

